# Discrete Choice Modeling for Travel Behavior Analysis: From Multinomial Logit to More Advanced Forms

## C2SMARTER Student Learning Hub Series

### Xiyuan Ren
### April 11, 2025

#### ---------------------

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import xlogit
import warnings
warnings.filterwarnings('ignore')
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
from math import radians, cos, sin, asin, sqrt
import pickle
import geopandas as gpd
import mapclassify
import webbrowser, pathlib

## Example 1: MNL, NL, MXL for Commute Choice Modeling 

Ren, X., & Chow, J. Y. (2022). A random-utility-consistent machine learning method to estimate agents’ joint activity scheduling choice from a ubiquitous data set. Transportation Research Part B: Methodological, 166, 396-418.

<img src="image/commute_choice.jpg" style="width:90%">

### 1.Data Structure

In [ ]:
Commuting_choice = pd.read_csv("Commuting_choice_0507.csv")

In [ ]:
Commuting_choice.head()

In [ ]:
print('Number of rows:',len(Commuting_choice))
print('Number of individuals:',len(Commuting_choice['iid'].unique()))
print('Number of choice observations:',int(len(Commuting_choice)/len(Commuting_choice['alternative'].unique())))
print('Number of alternatives:',len(Commuting_choice['alternative'].unique()))

In [ ]:
print(Commuting_choice['alternative'].unique())

### 2.Utility function
#### $$U_{ij}=\theta_{time}time_{commute}+\theta_{cost}cost_{commute}+\theta_{mode}mode_{commute}+\theta_{SDE}SDE+\theta_{SDL}SDL+\theta_{PL}PL+\theta_{duration}Dur_{work}+\epsilon_{i,j}$$ where:
$time_{commute}$: commuting travel time (vary across i,j)

$cost_{commute}$: commuting travel cost (vary across i,j)

$mode_{commute}$: commuting mode constant (vary across j)

$SDE$: schedule deviation--earlier than regular workplace arrival time (vary across i,j)

$SDL$: schedule deviation--later than regular workplace arrival time (vary across i,j)

$PL$: additional penalty for being late for work (vary across i,j)

$Dur_{work}$: total work duration (vary across i,j)

$\epsilon_{i,j}$: random disturbance following Gumbel distribution (vary across i for each j)

### 3.Estimate MNL and MXL using xlogit

Arteaga, C., Park, J., Beeramoole, P. B., & Paz, A. (2022). xlogit: An open-source Python package for GPU-accelerated estimation of Mixed Logit models. Journal of Choice Modelling, 42, 100339.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler()
Commuting_choice_ms = Commuting_choice.copy(deep=True)
Commuting_choice_ms.iloc[:,3:-1] = ms.fit_transform(Commuting_choice_ms.iloc[:,3:-1].values)

In [ ]:
from xlogit import MultinomialLogit, MixedLogit

#### In MNL, all parameters (theta) are assummed to be fixed values

In [ ]:
varnames = ['t_commute','c_commute','M_commute2','SDE_work','SDL_work','PL_work','ln_dwork']

MNL = MultinomialLogit()
MNL.fit(X=Commuting_choice_ms[varnames], y=Commuting_choice_ms['chosen'], varnames=varnames,
        ids=Commuting_choice_ms['iid'],alts=Commuting_choice_ms['alternative'])

MNL.summary()

$$\mathrm{Loglikelihood} \;=\; \sum_{i=1}^{N} \sum_{j=1}^{J} y_{ij}\,\ln P_{ij}$$
$$McFadden \space R^2 \;=\; 1 \;-\; \frac{LL(\beta)}{LL(0)}$$

In [ ]:
LL_MNL = MNL.loglikelihood
num_alt = len(Commuting_choice['alternative'].unique())
num_observation = int(len(Commuting_choice)/len(Commuting_choice['alternative'].unique()))
LL_0 = np.log(1/num_alt) * num_observation
print('McFadden R Square of MNL:',1-LL_MNL/LL_0)

#### In MXL, parameters are assumed to follow a parameteric distribution (e.g. normal, uniform, triangular)

In [ ]:
varnames = ['t_commute','c_commute','M_commute2','SDE_work','SDL_work','PL_work','ln_dwork']

MXL = MixedLogit()
MXL.fit(X=Commuting_choice_ms[varnames], y=Commuting_choice_ms['chosen'], varnames=varnames,
        ids=Commuting_choice_ms['iid'],alts=Commuting_choice_ms['alternative'],
        randvars={'t_commute':'n','c_commute':'n','M_commute2':'n','SDE_work':'n','SDL_work':'n','ln_dwork':'n'},
        n_draws=100)

MXL.summary()

In [ ]:
LL_MXL = MXL.loglikelihood
print('McFadden R Square of MXL:',1-LL_MXL/LL_0)

### 4.Estimate AMXL

#### In AMXL, each agent (an individual or a group of individuals) has a unique set of parameters

<img src="image/AMXL.jpg" style="width:40%">

In [ ]:
import AMXL_functions
import importlib
importlib.reload(AMXL_functions)
from AMXL_functions import solve_agent_commuting,One_iteration_AMXL

In [ ]:
alter_num_c = int(Commuting_choice_ms.groupby('iid').agg({'hw_od':'count'}).mean().values)
np.random.seed(8521)
epsilon_c = np.random.gumbel(0,1,26149*alter_num_c).reshape(26149,alter_num_c)

print('Individual 1')
iid = 560
aa = Commuting_choice_ms[Commuting_choice_ms['iid']==iid]
variable,Z = solve_agent_commuting(aa,[0,0,0,0,0,0,0],epsilon_c,iid=iid,safe_boundary=0.5)
print(pd.DataFrame(variable[None,:],columns=varnames))
print('------------------')

print('Individual 2')
iid = 132
aa = Commuting_choice_ms[Commuting_choice_ms['iid']==iid]
variable,Z = solve_agent_commuting(aa,[0,0,0,0,0,0,0],epsilon_c,iid=iid,safe_boundary=0.5)
print(pd.DataFrame(variable[None,:],columns=varnames))

### 5.Let's try 500 sample and compare MNL, MXL, AMXL

In [ ]:
sample_size = 500
data_sample = Commuting_choice_ms.iloc[:sample_size*num_alt]

shuffle = range(1,26150)
theta_0 = [0,0,0,0,0,0,0]
start_time = time.time()
theta_0, theta_i, sb_c = One_iteration_AMXL(data_sample, shuffle, epsilon_c, theta_0, 
                                           sample_size=sample_size,bound=30,boundary_max=3,boundary_min=1,step=0.4)
end_time = time.time()
print('Estimation time of AMXL per iteration: %.1f seconds'%(end_time-start_time))

In [ ]:
start_time = time.time()
MNL.fit(X=data_sample[varnames], y=data_sample['chosen'], varnames=varnames,
        ids=data_sample['iid'],alts=data_sample['alternative'])
end_time = time.time()
print('Estimation time of MNL: %.2f seconds'%(end_time-start_time))

In [ ]:
start_time = time.time()
MXL.fit(X=data_sample[varnames], y=data_sample['chosen'], varnames=varnames,
        ids=data_sample['iid'],alts=data_sample['alternative'],
        randvars={'t_commute':'n','c_commute':'n','M_commute2':'n','SDE_work':'n','SDL_work':'n','ln_dwork':'n'},
        n_draws=100)
end_time = time.time()
print('Estimation time of MXL: %.1f seconds'%(end_time-start_time))

In [ ]:
X = [Commuting_choice_ms[Commuting_choice_ms['iid']==iid][varnames].values for iid in range(1,sample_size+1)]
X = np.array(X)
X = np.transpose(X, (0, 2, 1)) # shape (sessions,attributes,alternatives)
Y = [Commuting_choice_ms[Commuting_choice_ms['iid']==iid]['chosen'].values for iid in range(1,sample_size+1)]
Y = np.array(Y)

V = (X * theta_i[:,:,None]).sum(axis=1)
V = V - V.min(axis=1)[:,None]
demo = np.exp(V).sum(axis=1).reshape(X.shape[0],1)
P = np.exp(V) / demo
LL_0 = np.log(1/num_alt) * sample_size

LL_MNL = MNL.loglikelihood
print('McFadden R Square of MNL:',1-LL_MNL/LL_0)
LL_MXL = MXL.loglikelihood
print('McFadden R Square of MXL:',1-LL_MXL/LL_0)
LL_AMXL = -log_loss(Y, P, normalize=False)
print('McFadden R Square of AMXL:',(1 - LL_AMXL/LL_0))

### 6.Overfitting Issues in AMXL

<img src="image/AMXL_out_of_sample_accuracy.jpg" style="width:90%">

# Example 2: Group-level AMXL for Statewide Mode Choice

Ren, X., Chow, J. Y., Bansal P. (2025). Estimating a k-modal nonparametric mixed logit model with market-level data, Transportation Research Part B: Methodological, accepted. https://arxiv.org/abs/2309.13159

<img src="image/OD_data.jpg" style="width:90%">

### 1.Data Structure

In [ ]:
with open('X_all.pickle', 'rb') as handle:
    X = pickle.load(handle)
with open('Y_all.pickle', 'rb') as handle:
    Y = pickle.load(handle)
with open('num_all.pickle', 'rb') as handle:
    num = pickle.load(handle)
with open('id_all.pickle', 'rb') as handle:
    group_id = pickle.load(handle)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
var_name = ['auto_tt','transit_ivt','transit_at','transit_et','transit_nt','non_vehicle_tt',
              'cost','constant_driving','constant_transit','constant_ondemand','constant_biking','constant_walking']
mode_name = ['Driving','Transit','On-demand','Biking','Walking','Carpool']

In [ ]:
gid = 100
print('Market/Group ID:',group_id[gid])
print('Number of trips per day:',num[gid])

In [ ]:
pd.DataFrame(X[gid].T, columns=var_name, index=mode_name)

In [ ]:
pd.DataFrame(Y[gid].T, columns=['Mode Share'], index=mode_name)

### 2.Utility function
<div style="font-size: 130%;">
$$V_{driving,t}=\theta_{\text{\textit{auto-tt}},t}\text{\textit{Auto-tt}}_{driving,t}+\theta_{cost,t}Cost_{driving,t}+\theta_{asc-driving,t}$$
$$V_{transit,t}=\theta_{AT,t}\text{\textit{AT}}_{transit,t}+\theta_{ET,t}\text{\textit{ET}}_{transit,t}+\theta_{IVT,t}\text{\textit{IVT}}_{transit,t}+\theta_{NT,t}\text{\textit{NT}}_{transit,t}+\theta_{cost,t}Cost_{transit,t}+\theta_{asc-transit,t}$$
$$V_{ondemand,t}=\theta_{\text{\textit{auto-tt}},t}{\textit{Auto-tt}}_{ondemand,t}+\theta_{cost,t}Cost_{ondemand,t}+\theta_{asc-ondemand,t}$$  
$$V_{biking,t}=\theta_{\text{\textit{non-auto-tt}},t}{\textit{Non-auto-tt}}_{biking,t} +\theta_{asc-biking,t}$$ 
$$V_{walking,t}=\theta_{\text{\textit{non-auto-tt}},t}{\textit{Non-auto-tt}}_{walking,t} +\theta_{asc-walking,t}$$
$$V_{carpool,t}=\theta_{\text{\textit{auto-tt}},t}\text{\textit{Auto-tt}}_{carpool,t}+\theta_{cost,t}Cost_{carpool,t}$$
</div>

where:

$\text{\textit{Auto-tt}}$: Auto travel time (unit: hour)

$AT$: Transit access time (unit:hour)

$ET$: Transit egress time (unit:hour)

$IVT$: Transit in-vehicle time (unit:hour)

$NT$: Transit number of transfers

$Cost$: Travel cost/ trip fare (unit:$)

### 3. Estimate Group-level AMXL

In [ ]:
import AMXL_functions
import importlib
importlib.reload(AMXL_functions)
from AMXL_functions import group_level_IO

In [ ]:
var_name = ['auto_tt','transit_ivt','transit_at','transit_et','transit_nt','non_vehicle_tt',
              'cost','constant_driving','constant_transit','constant_ondemand','constant_biking','constant_walking']
lb = [-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10]
ub = [0,0,0,0,0,0,0,10,10,10,10,10]

In [ ]:
gid = 3
Y_line = Y[gid,:]
X_line = X[gid,:,:]
theta_0 = np.zeros(len(var_name))
theta,Z,rho,mse,mae,LL,LL_0,P = group_level_IO(Y_line,X_line,theta_0,lb=lb,ub=ub,tol=0.1)
print('Market/Group ID:',group_id[gid])
print('Number of trips per day:',num[gid])
print('Mean absolute error per mode share: %.2f%%'%(mae*100))
print('McFadden R-square: %.4f'%(rho))
print('-----------')
print(pd.DataFrame(theta[None,:],columns=var_name,index=['Estimated Value']).round(4).T)

### 4. Explore the distribution of mode choice parameters

Estimated parameters for New York State: https://zenodo.org/records/8113817

In [ ]:
all_agents = gpd.read_file("shapefile/all_agents.shp")
all_agents.rename(columns={'origin_bgr':'origin_bgrp','destinatio':'destination_bgrp','length':'Trip_length'},inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))
all_agents.plot(ax=ax,linewidth=0.2)
ax.axis('off')
plt.show()

In [ ]:
with open('beta_array.pickle', 'rb') as handle:
    theta_i = pickle.load(handle)[:,:-1]

all_agents[var_name] = theta_i
all_agents['VOT'] = theta_i[:,0]/theta_i[:,6] # Value-of-time: theta_time/theta_cost|

In [ ]:
theta_result = pd.DataFrame(theta_i)
theta_result['group_id'] = group_id
NYC_county = ['36061','36047','36005','36081','36085']
region_marker = theta_result['group_id'].map(lambda x: (x.split('_')[0][:5] in NYC_county) & (x.split('_')[1][:5] in NYC_county))

In [ ]:
scheme = mapclassify.NaturalBreaks(all_agents[region_marker]['VOT'], k=5)
bins = list(scheme.bins)
cmap = ['#470057','#385194','#129188','#54d058','#fff000']

def VOT_plotter(all_agents,region_marker,column_name='VOT',segment='All',fixed_group=False):
    if segment=='All':
        data = all_agents[all_agents['VOT']<200]
    else:
        data = all_agents[pd.Series(group_id).map(lambda x:x.split('_')[-1])==segment]
    data2 = data[region_marker]
    data = data[data[column_name]<=data2[column_name].max()]
    # define scheme
    cmap = ['#470057','#385194','#129188','#54d058','#fff000']
    if fixed_group:
        bins = [10,25,50,75,data[column_name].max()]
        bins2 = [10,25,50,75,data2[column_name].max()]
    else:
        scheme = mapclassify.NaturalBreaks(data[column_name], k=5)
        bins = list(scheme.bins)
        scheme2 = mapclassify.NaturalBreaks(data2[column_name], k=5)
        bins2 = list(scheme2.bins)
    #plot
    fig,ax = plt.subplots(1,2,figsize=(15,8), gridspec_kw={'width_ratios': [1.27,1]})
    data[(data[column_name]<bins[0])].plot(linewidth=0.5,ax=ax[0],color=cmap[0],label='%.2f-%.2f $/hour'%(data[column_name].min(),bins[0]))
    for i in range(1,5):
        data[(data[column_name]<bins[i])&(data[column_name]>=bins[i-1])].plot(linewidth=0.5,ax=ax[0],color=cmap[i],label='%.2f-%.2f $/hour'%(bins[i-1],bins[i]))
    ax[0].set_title('Value of time (VOT) in New York State',fontsize=16)
    ax[0].axis('off')
    ax[0].legend(loc='lower left')
    data2[(data2[column_name]<bins2[0])].plot(linewidth=0.5,ax=ax[1],color=cmap[0],label='%.2f-%.2f $/hour'%(data2[column_name].min(),bins2[0]))
    for i in range(1,5):
        data2[(data2[column_name]<bins2[i])&(data2[column_name]>=bins2[i-1])].plot(linewidth=0.5,ax=ax[1],color=cmap[i],label='%.2f-%.2f $/hour'%(bins2[i-1],bins2[i]))
    ax[1].set_title('Value of time (VOT) in New York City',fontsize=16)
    ax[1].axis('off')
    ax[1].legend(loc='upper left')
    plt.subplots_adjust(wspace=0.1, hspace=0.1)

In [ ]:
VOT_plotter(all_agents,region_marker,column_name='VOT',segment='All',fixed_group=True)

In [ ]:
all_agents['VOT'].hist(bins=100,figsize=(6,4))
plt.xlim([-20,200])

In [ ]:
scheme = mapclassify.NaturalBreaks(all_agents[region_marker]['VOT'], k=5)
bins = list(scheme.bins)
cmap = ['#470057','#385194','#129188','#54d058','#fff000']

def coeff_plotter(all_agents,region_marker,column_name='constant_driving',segment='All',fixed_group=False):
    if segment=='All':
        data = all_agents.copy()
    else:
        data = all_agents[pd.Series(group_id).map(lambda x:x.split('_')[-1])==segment]
    data2 = data[region_marker]
    data = data[data[column_name]<=data2[column_name].max()]
    # define scheme
    cmap = ['#470057','#40448d','#28798c','#22a887','#78d348','#fff000']
    if fixed_group:
        bins = [-2,-1,0,1,2,data[column_name].max()]
        bins2 = [-2,-1,0,1,2,data[column_name].max()]
    else:
        scheme = mapclassify.Quantiles(data[column_name], k=6)
        bins = list(scheme.bins)
        scheme2 = mapclassify.Quantiles(data2[column_name], k=6)
        bins2 = list(scheme2.bins)
    #plot
    fig,ax = plt.subplots(1,2,figsize=(15,8), gridspec_kw={'width_ratios': [1.27,1]})
    data[(data[column_name]<bins[0])].plot(linewidth=0.5,ax=ax[0],color=cmap[0],label='%.2f-%.2f'%(data[column_name].min(),bins[0]))
    for i in range(1,6):
        data[(data[column_name]<bins[i])&(data[column_name]>=bins[i-1])].plot(linewidth=0.5,ax=ax[0],color=cmap[i],label='%.2f-%.2f'%(bins[i-1],bins[i]))
    ax[0].set_title('%s in New York City'%(column_name.capitalize()),fontsize=16)
    ax[0].axis('off')
    ax[0].legend(loc='lower left')
    data2[(data2[column_name]<bins2[0])].plot(linewidth=0.5,ax=ax[1],color=cmap[0],label='%.2f-%.2f'%(data2[column_name].min(),bins2[0]))
    for i in range(1,6):
        data2[(data2[column_name]<bins2[i])&(data2[column_name]>=bins2[i-1])].plot(linewidth=0.5,ax=ax[1],color=cmap[i],label='%.2f-%.2f'%(bins2[i-1],bins2[i]))
    ax[1].set_title('%s in New York City'%(column_name.capitalize()),fontsize=16)
    ax[1].axis('off')
    ax[1].legend(loc='upper left')
    plt.subplots_adjust(wspace=0.1, hspace=0.1)

In [ ]:
coeff_plotter(all_agents,region_marker,column_name='constant_transit',fixed_group=True)

In [ ]:
coeff_plotter(all_agents,region_marker,column_name='constant_walking',fixed_group=True)

In [ ]:
coeff_plotter(all_agents,region_marker,column_name='constant_biking',fixed_group=True)

### 5. How these can be integrated into an optimization model?

Ren, X., Chow, J. Y., & Guan, C. (2024). Mobility service design with equity-aware choice-based decision-support tool: New York case study. Transportation Research Part D: Transport and Environment, 132, 104255.

#### Assuming there will be two new mobility services in New York State, namely, service A and service B. 
- Service A is a personalized ride-hailing service with shorter travel time and higher trip fare.
- Service B is an on-demand microtransit service with longer travel time and lower trip fare.
#### How to deploy these services throughout the New York State, given a budget and an objective?

<img src="image/system_optimization.jpg" style="width:70%">

In [ ]:
file_path = pathlib.Path("html/multi_Objective1_20_5000.html")   # relative path
webbrowser.open(file_path.resolve().as_uri())

In [ ]:
file_path = pathlib.Path("html/multi_Objective4_20_5000.html")   # relative path
webbrowser.open(file_path.resolve().as_uri())